<a href="https://colab.research.google.com/github/gagan3012/keytotext/blob/master/notebooks/Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keytotext --upgrade
!pip install datasets

     |████████████████████████████████| 819kB 37.5MB/s 
     |████████████████████████████████| 2.5MB 41.6MB/s 
     |████████████████████████████████| 1.8MB 40.0MB/s 
     |████████████████████████████████| 1.2MB 40.6MB/s 
     |████████████████████████████████| 10.6MB 49.0MB/s 
     |████████████████████████████████| 645kB 44.7MB/s 
     |████████████████████████████████| 122kB 58.5MB/s 
     |████████████████████████████████| 829kB 53.5MB/s 
     |████████████████████████████████| 235kB 54.1MB/s 
     |████████████████████████████████| 901kB 41.2MB/s 
     |████████████████████████████████| 3.3MB 48.6MB/s 
     |████████████████████████████████| 174kB 50.5MB/s 
     |████████████████████████████████| 102kB 14.6MB/s 
     |████████████████████████████████| 133kB 54.0MB/s 
     |████████████████████████████████| 1.3MB 46.1MB/s 
     |████████████████████████████████| 71kB 11.8MB/s 
     |████████████████████████████████| 296kB 42.4MB/s 
     |████████████████████████████████| 143kB 54

In [13]:
from datasets import load_dataset

dataset = load_dataset('common_gen',split='train')

Using custom data configuration default


Dataset common_gen downloaded and prepared to /root/.cache/huggingface/datasets/common_gen/default/2020.5.30/1a9e8bdc026c41ce7a9e96260debf7d2809cb7fd63fa02b017e4fac1b00c6b23. Subsequent calls will reuse this data.


In [14]:
print(dataset)

Dataset({
    features: ['concept_set_idx', 'concepts', 'target'],
    num_rows: 67389
})


In [15]:
import pandas as pd

df = pd.DataFrame()
df['keywords']=dataset['concepts']
df['text'] = dataset['target']
df.head()

,keywords,text
0,"[ski, mountain, skier]",Skier skis down the mountain
1,"[ski, mountain, skier]",A skier is skiing down a mountain.
2,"[ski, mountain, skier]",Three skiers are skiing on a snowy mountain.
3,"[wag, tail, dog]",The dog is wagging his tail.
4,"[wag, tail, dog]",A dog wags his tail at the boy.


In [5]:
def clean(keywords):
  source_text = ' '.join(map(str, keywords))
  return source_text

In [6]:
df['keywords'] = df['keywords'].apply(lambda x: clean(x))
df.head()

,keywords,text
0,ski mountain skier,Skier skis down the mountain
1,ski mountain skier,A skier is skiing down a mountain.
2,ski mountain skier,Three skiers are skiing on a snowy mountain.
3,wag tail dog,The dog is wagging his tail.
4,wag tail dog,A dog wags his tail at the boy.


In [7]:
from sklearn.model_selection import train_test_split
train_df,test_df = train_test_split(df,test_size=0.01, random_state=42)

In [8]:
from keytotext import trainer

model = trainer()
model.from_pretrained(model_name="t5-small")
model.train(train_df = train_df[:2000],test_df=test_df,batch_size=4, max_epochs=3, use_gpu=True)
model.upload(hf_username="gagan3012",model_name="k2t-test3")

Global seed set to 42


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc



  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Global seed set to 42


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: LightningDeprecationWarning: Relying on `self.log('val_loss', ...)` to set the ModelCheckpoint monitor is deprecated in v1.2 and will be removed in v1.4. Please, create your own `mc = ModelCheckpoint(monitor='your_monitor')` and use it as `Trainer(callbacks=[mc])`.
  "Relying on `self.log('val_loss', ...)` to set the ModelCheckpoint monitor is deprecated in v1.2"



Enter your HuggingFace password··········


HTTPError: ignored

In [ ]:
model.load_model("/content/outputs/keytotext-epoch-3", use_gpu=True)

keywords=["ski", "mountain", "sky"]
model.predict(keywords)

'a sky on the mountain of skis.'